# Do's and Don'ts

Let's cover some patterns and antipatterns to help you improve your designs.

## Functions

### Don't put functions in functions

Functions _can_ be defined in other functions in Python. But because you can do something does not mean you should.

In [2]:
def my_confusing_function():
    something_important = 42
    
    def inner_function(x):
        return x * something_important
    
    something_important = 7
    
    return inner_function(2)

In [3]:
print(my_confusing_function())

14


But this is _bad_, don't do it. For one, this breaks the flow of the function and adds unneccisary nesting. When reading a function definition, you should see code, not more function definitions. More importantly, it breaks encapuslation; a function in a function _can_ capture the outer function's scope. When you place a function within a function, the reader has to go through it, line by line, and see what variables are captured, if any. And Python’s scoping rules are evil - even the author of Python finds them hard sometimes [(see his recent tweet)](https://twitter.com/gvanrossum/status/1354305179244392453?s=20) (classes in functions or classes are _really_ bad due to some weird scoping rules for classes - hoping I don't need to tell you that!).

There is one valid place to do a function in a function - that's for wrappers, specifically decorators. But that is the _point_ of the outer function; it's not doing anything else but wrapping an inner function. And, in general, they are often more readable as classes with a `__call__` anyway, so you might not even want the function-in-function form even in this case.

There are _very_ rare cases where you might want to define a function inside a function if you specifically _have_ to capture a variable - but you never _really_ have to capture a variable - use `functools.partial` instead if you have an API that takes a function and you need to provide the right signature. 

A one line "named lambda" function _might_ be an exception - but I think it’s still clearer to make it a helper. Use underscore names if you want to indicate that it’s not for external use. This is Python specific - C++ lambdas have an explicit capture specification that is a lot better.

If you want a private helper function, just put an underscore at the front of the helper function.

Here's a possible rewrite for the example above:

In [8]:
def _helper_function(something_important, x):
    return x * something_important

def my_better_function():
    something_important = 42
      
    something_important = 7
    
    return _helper_function(something_important, 2)

In [11]:
print(my_better_function())

14


What if you had an API that required a function with one variable? Use `functools.partial` or a lambda for the capture, not a function in a function. It's much clearer that you are doing something speical, and the interface to the helper function is still completely explicit, with no hidden capture:

In [14]:
import functools

def _helper_function(something_important, x):
    return x * something_important

def my_other_function():
    something_important = 42
      
    inner_function = functools.partial(_helper_function, something_important)
    
    something_important = 7
    
    return inner_function(2)

print(my_other_function())

84


Notice how this actually prints the value exactly at the point of capture, rather than having a shared namespace. The API of `_helper_function` is still completely explicit, with no hidden capture; you don't need to look around to see what it takes as arguments, you just look at the argument list.

### Don't have giant functions

Don’t have mega-functions that do too much. Factor out the work as much as you can. It's better for unit testing, it's better for reasoning what each piece does, it is easier to read smaller chunks. Don't go over board and make a ton of one line functions; look for coherent, clear concepts. 10-20 lines should be the max for most functions. 

### Do consider using classes instead of functions
If you have something really complicated, it’s okay to use a class. You want to offload structural work to the language; then the type checker can help you with it, etc. If you find yourself needing to capture often or have long argument lists (internally), this might be a much more explicit, readable, and clear alternative.

## Types

## Do factor into common types

Try to normalize to a common type instead of handling differing types everywhere. If you support a list of items or a single idem, write a function that works on lists of items, then call with with `[item]` if a single item is passed. The quicker you reduce a complicated Union of types down to a single type, the better.

Also, try to avoid using tightly constrainted types; when typing, try to accept the most general type. This exposes a common issue; if your "item" is Iterable, you'll have to be very specific and differenciate between lists and other types of iterables; this is generally bad API and should be avoided if possible. It is baked into a few places, including `__getitem__` for many classes; NumPy differenciates between `a[(1,2)]` and `a[[1,2]]`. But try to avoid making a similar mistake if possible.

## Other

## Do use lots of files

Break things up into more files. Files should have roughly one class and its helpers, or one major function and its helpers, or a collection of related functions. Then you can import from it. You'll often end up with a `utils.py`, but don't put everything into it. Remember: programming is about orginization, and your file structure is an important part of that orginization. If you have trouble navigating around, invest some time in a good editor. Even the GitHub viewer now jumps to definitions when you click Python functions, and PyCharm, Visual Studio Code, and more have fast navigation; classic editors have plugins that help with navigation too.